# **Agentic Recommendation System (POC)**
### Hybrid Collaborative Filtering + Semantic Search with Critic-Loop Self-Correction

This project implements an **Agentic Workgraph** for movie recommendations. Unlike traditional recommenders that use static formulas, this system uses a "Critic" agent to identify genre drift and popularity bias, automatically adjusting retrieval weights in real-time.

## System Architecture
The pipeline is orchestrated via a **StateGraph**:
1. **Intent Agent**: Decodes user natural language into retrieval goals.
2. **Planner Agent**: Sets the initial $w_{cf}$ (Collaborative) and $w_{sem}$ (Semantic) weights.
3. **Retrieval Tooling**:
   - `SimpleCFRecommender`: Item-Item similarity based on MovieLens 100k.
   - `Semantic Index`: FAISS-based vector search for "vibe" and "context."
4. **Ranker v2 (Advantage-Weighted)**: Penalizes popularity bias to surface niche "hidden gems."
5. **Critic Agent**: Checks for failures (e.g., suggesting "cheesy" movies when user asked for "real").

## Modular Components
- `orchestrator.py`: The LangGraph controller.
- `ranker_v2.py`: Implementation of Advantage-Weighted retrieval.
- `tools/cf_tool.py`: Scalable item-item similarity engine.
- `config.py`: Centralized hyperparameters (Novelty Lambda, Advantage Alpha).

## Performance Insights
This system successfully solves the **"Similarity Drift"** problem. By utilizing a 50/50 hybrid weight, the system provides a balanced list that respects both the user's specific text query and the historical behavior of similar users.

## Getting Started
1. Install requirements: `pip install pandas numpy torch langgraph openai`
2. Configure your `OPENAI_API_KEY` in environment variables.
3. Run the orchestrator script to see the agent trace.

In [1]:
!pip install hypercorn requests --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.5 MB/s eta 0:00:00


In [2]:
!mkdir -p agents tools graph data embeddings api rankers notebooks
!touch agents/__init__.py tools/__init__.py graph/__init__.py api/__init__.py

In [3]:
# 1. Install specific versions to avoid NumPy 2.0 breaking SAR
!pip install -q --force-reinstall numpy==1.26.4 scipy==1.11.4 pandas==2.1.4

!pip install -q \
  recommenders \
  scikit-learn tqdm requests \
  langgraph langchain langchain-community langchain-core \
  transformers accelerate bitsandbytes sentencepiece \
  sentence-transformers \
  hnswlib

# 2. Print versions to verify
import numpy as np
import pandas as pd
print(f"Current NumPy: {np.__version__} (Should be 1.26.4)")
print(f"Current Pandas: {pd.__version__}")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:0

In [4]:
from google.colab import userdata
import os

OPENAI_KEY = userdata.get("OPENAI_API_KEY")

if not OPENAI_KEY:
    raise RuntimeError(
        "OPENAI_API_KEY not found. "
        "Check Colab → Secrets → OPENAI_API_KEY and restart runtime."
    )
os.environ["OPENAI_API_KEY"] = OPENAI_KEY
print(" OpenAI API key loaded from Colab secrets")

 OpenAI API key loaded from Colab secrets


In [5]:
%%writefile config.py
from dataclasses import dataclass
import torch


@dataclass
class POCConfig:
    # -----------------------------
    # Data
    # -----------------------------
    data_dir: str = "data/ml-latest-small"
    movielens_url: str = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

    # Embeddings (persisted)

    embedding_model_id: str = "text-embedding-3-small"
    embedding_dim: int = 1536
    embedding_path: str = "embeddings/movie_embeddings.npy"
    embedding_ids_path: str = "embeddings/movie_ids.json"

    # Retrieval sizes

    cf_k: int = 200
    semantic_k: int = 80
    final_k: int = 20

    llm_model_id: str = "microsoft/phi-2"
    llm_max_new_tokens: int = 160
    llm_temperature: float = 0.2

    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # Verbosity

    verbose: bool = True
    verbose_prompts: bool = False
    verbose_openai: bool = True

    # Strategy mode

    # v1 = baseline hybrid
    # v2 = agentic + critic + advantage-weighted
    strategy_version: str = "v2"

    # Planner constraints

    enforce_hybrid_for_search: bool = True

    # CF should never dominate semantic intent for search queries
    min_cf_weight: float = 0.20
    max_cf_weight: float = 0.60

    # Advantage-Weighted Ranking
    # How strongly we favor "advantaged" items over baseline popularity
    advantage_alpha: float = 0.6

    # Penalizes popularity bias (higher = more niche)
    novelty_lambda: float = 0.20

    critic_topn: int = 10

    # Popularity control
    popularity_mean_threshold: float = 0.65

    # Minimum unique genres in top-N
    genre_diversity_min_unique: int = 4

    min_primary_genre_ratio: float = 0.40

    # Amount to reduce CF when genre drift is detected
    genre_drift_cf_penalty: float = -0.30

    # Corresponding semantic boost
    genre_drift_semantic_boost: float = 0.30

    prefer_weight_adjustments: bool = True

    allow_hard_removal_without_weight_shift: bool = False


Writing config.py


In [6]:
%%writefile tools/item_stats.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Any, Optional
import numpy as np
import pandas as pd


@dataclass
class ItemStats:
    popularity_norm: Dict[int, float]
    avg_rating_norm: Dict[int, float]
    rating_count: Dict[int, int]

    @staticmethod
    def build(ratings: pd.DataFrame) -> "ItemStats":
        # counts
        counts = ratings.groupby("movieId")["rating"].count().astype(int)
        max_c = int(counts.max()) if len(counts) else 1
        popularity_norm = {int(mid): float(c / max_c) for mid, c in counts.items()}

        # avg rating
        avgs = ratings.groupby("movieId")["rating"].mean()
        # Normalize ratings from [0.5..5] -> [0..1] (MovieLens uses 0.5 increments)
        avg_rating_norm = {int(mid): float((r - 0.5) / (5.0 - 0.5)) for mid, r in avgs.items()}

        rating_count = {int(mid): int(c) for mid, c in counts.items()}
        return ItemStats(popularity_norm=popularity_norm, avg_rating_norm=avg_rating_norm, rating_count=rating_count)

    def get_popularity(self, movie_id: int) -> float:
        return float(self.popularity_norm.get(int(movie_id), 0.0))

    def get_avg_rating(self, movie_id: int) -> float:
        return float(self.avg_rating_norm.get(int(movie_id), 0.5))

    def as_debug_dict(self, movie_id: int) -> Dict[str, Any]:
        mid = int(movie_id)
        return {
            "movieId": mid,
            "popularity_norm": self.get_popularity(mid),
            "avg_rating_norm": self.get_avg_rating(mid),
            "rating_count": int(self.rating_count.get(mid, 0)),
        }


Writing tools/item_stats.py


In [7]:
%%writefile tools/data_loader.py
import os
import zipfile
import requests
from io import BytesIO
from typing import Tuple

import pandas as pd
from config import POCConfig


class MovieLensLoader:
    """
    Responsible for loading the MovieLens dataset in a safe, repeatable way.

    Design principles:
    - Download only if data is missing
    - Never re-download unnecessarily
    - Fail loudly if data is corrupted
    - Return clean, ready-to-use DataFrames
    """

    def __init__(self, cfg: POCConfig):
        self.cfg = cfg

    def _dataset_exists(self) -> bool:
        """
        Check whether the expected MovieLens files already exist.
        """
        ratings_path = os.path.join(self.cfg.data_dir, "ratings.csv")
        movies_path = os.path.join(self.cfg.data_dir, "movies.csv")
        return os.path.exists(ratings_path) and os.path.exists(movies_path)

    def _download_and_extract(self) -> None:
        """
        Download and extract the MovieLens dataset.
        """
        print("MovieLens dataset not found. Downloading...")

        response = requests.get(self.cfg.movielens_url, timeout=120)
        response.raise_for_status()

        with zipfile.ZipFile(BytesIO(response.content)) as z:
            z.extractall("data")

        print("Download and extraction complete.")

    def load(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Load ratings and movies DataFrames.

        This method is safe to call multiple times.
        """
        if not self._dataset_exists():
            self._download_and_extract()
        else:
            print("MovieLens dataset found locally. Skipping download.")

        ratings_path = os.path.join(self.cfg.data_dir, "ratings.csv")
        movies_path = os.path.join(self.cfg.data_dir, "movies.csv")

        try:
            ratings = pd.read_csv(ratings_path)
            movies = pd.read_csv(movies_path)
        except Exception as e:
            raise RuntimeError(
                "Failed to load MovieLens CSV files. "
                "The dataset may be corrupted."
            ) from e

        required_rating_cols = {"userId", "movieId"}
        required_movie_cols = {"movieId", "title"}

        if not required_rating_cols.issubset(ratings.columns):
            raise ValueError("ratings.csv is missing required columns")

        if not required_movie_cols.issubset(movies.columns):
            raise ValueError("movies.csv is missing required columns")

        return ratings, movies


Writing tools/data_loader.py


In [8]:
%%writefile rankers/ranker_v1.py
from __future__ import annotations
from typing import Any, Dict, List
import pandas as pd


class RankerV1:
    """
    Baseline v1 ranker:
    - linear fusion of normalized CF and semantic scores
    - no advantage weighting
    """

    def __init__(self, cfg, movie_map: Dict[int, Dict[str, str]]):
        self.cfg = cfg
        self.movie_map = movie_map

    def __call__(self, state: Dict[str, Any]) -> List[Dict[str, Any]]:
        cf = state.get("cf", pd.DataFrame())
        sem = pd.DataFrame(state.get("sem", []))

        if cf is None or isinstance(cf, list):
            cf = pd.DataFrame(cf)
        if sem is None:
            sem = pd.DataFrame()

        if cf.empty and sem.empty:
            return []

        if not cf.empty and not sem.empty:
            df = cf.merge(sem, on="movieId", how="outer").fillna(0.0)
        elif not cf.empty:
            df = cf.assign(semantic_score=0.0)
        else:
            df = sem.assign(cf_score=0.0)

        for col in ["cf_score", "semantic_score"]:
            mx = float(df[col].max()) if len(df) else 0.0
            if mx > 0:
                df[col] = df[col] / mx

        w_cf = float(state["plan"].get("weight_cf", 0.5))
        w_sem = float(state["plan"].get("weight_semantic", 0.5))
        df["score"] = w_cf * df["cf_score"] + w_sem * df["semantic_score"]

        df = df.sort_values("score", ascending=False).head(self.cfg.final_k)

        recs: List[Dict[str, Any]] = []
        for r in df.itertuples(index=False):
            mid = int(r.movieId)
            meta = self.movie_map.get(mid, {"title": "Unknown", "genres": ""})
            recs.append(
                {
                    "movieId": mid,
                    "title": meta.get("title", "Unknown"),
                    "genres": meta.get("genres", ""),
                    "score": float(r.score),
                    "signals": {
                        "cf": float(getattr(r, "cf_score", 0.0)),
                        "semantic": float(getattr(r, "semantic_score", 0.0)),
                    },
                }
            )
        return recs


Writing rankers/ranker_v1.py


In [9]:
%%writefile rankers/ranker_v2.py
from __future__ import annotations
from typing import Any, Dict, List
import pandas as pd
from tools.item_stats import ItemStats


class RankerV2:
    """
    Ranker v2: Advantage-Weighted Collaborative Retrieval (POC approximation)

    Steps:
    1) Build candidate pool from CF + semantic (already in state)
    2) Normalize signals
    3) Compute utility = w_cf*cf + w_sem*semantic
    4) baseline = popularity_norm (proxy for "expected utility")
    5) advantage = utility - advantage_alpha * baseline
    6) novelty_boost = novelty_lambda * (1 - popularity_norm)
    7) final = advantage + novelty_boost
    """

    def __init__(self, cfg, movie_map: Dict[int, Dict[str, str]], item_stats: ItemStats):
        self.cfg = cfg
        self.movie_map = movie_map
        self.item_stats = item_stats

    def __call__(self, state: Dict[str, Any]) -> List[Dict[str, Any]]:
        cf = state.get("cf", pd.DataFrame())
        sem = pd.DataFrame(state.get("sem", []))

        if cf is None or isinstance(cf, list):
            cf = pd.DataFrame(cf)
        if sem is None:
            sem = pd.DataFrame()

        if cf.empty and sem.empty:
            return []

        if not cf.empty and not sem.empty:
            df = cf.merge(sem, on="movieId", how="outer").fillna(0.0)
        elif not cf.empty:
            df = cf.assign(semantic_score=0.0)
        else:
            df = sem.assign(cf_score=0.0)

        # normalize signals
        for col in ["cf_score", "semantic_score"]:
            mx = float(df[col].max()) if len(df) else 0.0
            if mx > 0:
                df[col] = df[col] / mx

        plan = state.get("plan", {}) or {}
        w_cf = float(plan.get("weight_cf", 0.4))
        w_sem = float(plan.get("weight_semantic", 0.6))

        # utility = hybrid relevance / preference signal
        df["utility"] = w_cf * df["cf_score"] + w_sem * df["semantic_score"]

        # baseline = popularity_norm proxy (expected utility)
        df["baseline"] = df["movieId"].apply(lambda mid: self.item_stats.get_popularity(int(mid)))

        # advantage + novelty
        alpha = float(self.cfg.advantage_alpha)
        novelty_lambda = float(self.cfg.novelty_lambda)

        df["advantage"] = df["utility"] - alpha * df["baseline"]
        df["novelty_boost"] = novelty_lambda * (1.0 - df["baseline"])
        df["score"] = df["advantage"] + df["novelty_boost"]

        df = df.sort_values("score", ascending=False).head(self.cfg.final_k)

        recs: List[Dict[str, Any]] = []
        for r in df.itertuples(index=False):
            mid = int(r.movieId)
            meta = self.movie_map.get(mid, {"title": "Unknown", "genres": ""})
            recs.append(
                {
                    "movieId": mid,
                    "title": meta.get("title", "Unknown"),
                    "genres": meta.get("genres", ""),
                    "score": float(r.score),
                    "signals": {
                        "cf": float(getattr(r, "cf_score", 0.0)),
                        "semantic": float(getattr(r, "semantic_score", 0.0)),
                        "utility": float(getattr(r, "utility", 0.0)),
                        "baseline_popularity": float(getattr(r, "baseline", 0.0)),
                        "advantage": float(getattr(r, "advantage", 0.0)),
                        "novelty_boost": float(getattr(r, "novelty_boost", 0.0)),
                    },
                }
            )
        return recs


Writing rankers/ranker_v2.py


In [10]:
%%writefile tools/cf_tool.py
import numpy as np
import pandas as pd
from typing import Dict, Optional


class SimpleCFRecommender:
    """
    Deterministic item-item collaborative filtering recommender.

    Design goals:
    - Stable across runs
    - Fully explainable
    - No stochastic components
    - Safe fallbacks for unknown users
    """

    def __init__(self):
        self.users = None
        self.items = None
        self.user_idx: Dict[int, int] = {}
        self.item_idx: Dict[int, int] = {}
        self.idx_item: Dict[int, int] = {}
        self.sim: Optional[np.ndarray] = None
        self.mat: Optional[np.ndarray] = None

    def fit(self, ratings: pd.DataFrame) -> None:
        """
        Build an implicit user–item interaction matrix and
        compute item–item cosine similarity.
        """
        self.users = ratings["userId"].unique()
        self.items = ratings["movieId"].unique()

        self.user_idx = {int(u): i for i, u in enumerate(self.users)}
        self.item_idx = {int(m): i for i, m in enumerate(self.items)}
        self.idx_item = {i: m for m, i in self.item_idx.items()}

        interaction_matrix = np.zeros(
            (len(self.users), len(self.items)),
            dtype=np.float32
        )

        for row in ratings.itertuples(index=False):
            u = int(row.userId)
            m = int(row.movieId)
            interaction_matrix[self.user_idx[u], self.item_idx[m]] = 1.0

        item_matrix = interaction_matrix.T
        norms = np.linalg.norm(item_matrix, axis=1, keepdims=True) + 1e-8
        item_matrix = item_matrix / norms

        self.sim = item_matrix @ item_matrix.T
        self.mat = interaction_matrix

    def recommend(self, user_id: int, k: int = 50) -> pd.DataFrame:
        """
        Recommend items for a user based on item–item similarity.

        Always returns a valid DataFrame.
        """
        if self.mat is None or self.sim is None:
            raise RuntimeError("CF model not fitted. Call fit() first.")

        if user_id not in self.user_idx:
            return pd.DataFrame(columns=["movieId", "cf_score"])

        user_vector = self.mat[self.user_idx[user_id]]
        scores = self.sim @ user_vector

        top_indices = np.argsort(-scores)[:k]

        return pd.DataFrame({
            "movieId": [int(self.idx_item[i]) for i in top_indices],
            "cf_score": scores[top_indices].astype(float)
        })


Writing tools/cf_tool.py


In [11]:
%%writefile tools/semantic_tool.py
import os, json
import numpy as np
import hnswlib
from openai import OpenAI
from config import POCConfig

class SemanticSearchTool:
    def __init__(self, cfg: POCConfig):
        self.cfg = cfg
        self.client = OpenAI()
        self.index = None

    def _embed(self, texts):
        r = self.client.embeddings.create(
            model=self.cfg.embedding_model_id,
            input=texts
        )
        X = np.array([d.embedding for d in r.data], dtype=np.float32)
        X /= np.linalg.norm(X, axis=1, keepdims=True) + 1e-8
        return X

    def build_or_load(self, movies):
        os.makedirs("embeddings", exist_ok=True)

        if os.path.exists(self.cfg.embedding_path):
            print("Loading cached embeddings")
            X = np.load(self.cfg.embedding_path)
            self.movie_ids = json.load(open(self.cfg.embedding_ids_path))
        else:
            print("Building embeddings (one-time)")
            texts = (movies.title + " | " + movies.genres).fillna("").tolist()
            X = np.vstack([self._embed(texts[i:i+100]) for i in range(0, len(texts), 100)])
            np.save(self.cfg.embedding_path, X)
            self.movie_ids = movies.movieId.astype(int).tolist()
            json.dump(self.movie_ids, open(self.cfg.embedding_ids_path, "w"))

        self.index = hnswlib.Index(space="cosine", dim=X.shape[1])
        self.index.init_index(len(X), ef_construction=200, M=16)
        self.index.add_items(X, np.arange(len(X)))
        self.index.set_ef(64)

    def search(self, query, k):
        qv = self._embed([query])
        labels, dist = self.index.knn_query(qv, k=k)
        return [
            {"movieId": self.movie_ids[i], "semantic_score": 1.0 - d}
            for i, d in zip(labels[0], dist[0])
        ]


Writing tools/semantic_tool.py


In [12]:
%%writefile agents/intent_agent.py
import json
from typing import Any, Dict

from agents.openai_client import OpenAIJSONClient


class IntentAgent:
    """
    OpenAI-backed intent and clarification agent.

    IMPORTANT:
    - Does NOT accept an external llm object (prevents JSON serialization issues).
    - Always returns JSON-safe primitives.
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.llm = OpenAIJSONClient(model=model, temperature=0.2)

    def run(self, context: Dict[str, Any]) -> Dict[str, Any]:
        system_prompt = """
You are an intent classification agent for a movie recommender.

You must follow these rules:
- Think privately; do NOT reveal chain-of-thought.
- Output ONLY valid JSON (no markdown, no commentary).
- Use only the provided context. Do not invent user preferences.

Your job:
1) classify the user’s intent
2) decide if a clarification question is needed
3) produce a short trace list with high-level reasons (no hidden reasoning)
"""

        user_prompt = f"""
Context (JSON):
{json.dumps(context, indent=2)}

Allowed intents (choose exactly one):
- "search": user has a concrete query phrase or specific attributes
- "explore": browsing / discovery
- "comfort": safer, familiar picks; low novelty tolerance
- "quick_watch": time-constrained; prefers shorter / low-commitment

Hard rules:
- If "query" is a non-empty string -> intent MUST be "search"
- If novelty_tolerance exists and < 0.3 -> intent MUST be "comfort" (unless query is present)
- If available_minutes exists and <= 45 -> intent SHOULD be "quick_watch" (unless query is present)

Clarification rules:
- Set needs_clarification=true ONLY if information is insufficient to satisfy the intent.
- If needs_clarification=true, provide exactly ONE question.

Return ONLY this JSON schema:
{{
  "intent": "search|explore|comfort|quick_watch",
  "confidence": 0.0,
  "needs_clarification": false,
  "clarification_question": "",
  "trace": ["reason_a", "reason_b"]
}}
"""

        schema_hint = {
            "intent": "search",
            "confidence": 0.8,
            "needs_clarification": False,
            "clarification_question": "",
            "trace": ["..."],
        }

        resp = self.llm.generate_json(system_prompt, user_prompt, force_json=True, schema_hint=schema_hint, max_retries=1)

        data = resp["data"] if resp["ok"] else {}
        trace = list(data.get("trace", [])) if isinstance(data.get("trace", []), list) else []
        trace += resp.get("trace", [])
        trace.append("intent_agent_openai")

        # --- HARD GUARDS (deterministic enforcement) ---
        query = (context.get("query") or "").strip()
        novelty = context.get("novelty_tolerance", None)
        minutes = context.get("available_minutes", None)

        intent = str(data.get("intent", "explore"))
        if query:
            intent = "search"
        else:
            if isinstance(novelty, (int, float)) and novelty < 0.3:
                intent = "comfort"
            elif isinstance(minutes, (int, float)) and minutes <= 45:
                intent = "quick_watch"

        try:
            confidence = float(data.get("confidence", 0.6))
        except Exception:
            confidence = 0.6
        confidence = max(0.0, min(1.0, confidence))

        needs_clarification = bool(data.get("needs_clarification", False))
        clarification_question = str(data.get("clarification_question", "") or "")

        # If low confidence, ask a single deterministic clarification question
        if confidence < 0.5 and not needs_clarification:
            needs_clarification = True

        if needs_clarification and not clarification_question:
            # One question that helps both semantic + taste alignment
            clarification_question = (
                "Do you want the sci-fi to lean more cerebral and philosophical, "
                "or more emotional and character-driven?"
            )
            trace.append("default_clarification_applied")

        return {
            "intent": intent,
            "confidence": confidence,
            "needs_clarification": needs_clarification,
            "clarification_question": clarification_question,
            "trace": trace,
        }


Writing agents/intent_agent.py


In [13]:
%%writefile agents/planner_agent.py
from __future__ import annotations
from typing import Any, Dict
from config import POCConfig
from agents.openai_client import OpenAIJSONClient


class PlannerAgent:
    """
    Planner v2 (research-aligned):
    - For search intent, do NOT disable CF if cfg.enforce_hybrid_for_search
    - Enforce weight_cf >= cfg.min_cf_weight when hybrid is enabled
    - Maintain strict schema and stable defaults
    """

    def __init__(self, cfg: POCConfig):
        self.cfg = cfg
        self.client = OpenAIJSONClient(
            model="gpt-4o-mini",
            temperature=0.2,
            verbose=bool(cfg.verbose_openai),
        )
        self.verbose = bool(cfg.verbose)

    def _log(self, msg: str):
        if self.verbose:
            print(f"[PlannerAgent] {msg}")

    def run(self, intent: str, context: Dict[str, Any]) -> Dict[str, Any]:
        intent = (intent or "explore").strip().lower()
        query = (context.get("query") or "").strip()

        system_prompt = (
            "You are a Netflix-style Strategy Planner for a movie recommender.\n"
            "You must output ONLY valid JSON.\n"
            "Do NOT include commentary.\n"
            "Do NOT reveal hidden reasoning.\n"
        )

        # Research rules encoded as explicit constraints:
        # When a query exists, semantic must be used.
        # When hybrid enforcement is enabled, CF must be non-zero even for search
        user_prompt = (
            f"Intent: {intent}\n"
            f"Query: {query}\n"
            f"Context JSON:\n{context}\n\n"
            "Return JSON with exactly these keys:\n"
            "{\n"
            '  "use_cf": boolean,\n'
            '  "use_semantic": boolean,\n'
            '  "weight_cf": number (0..1),\n'
            '  "weight_semantic": number (0..1),\n'
            '  "trace": array of strings\n'
            "}\n\n"
            "Rules:\n"
            "- If Query is non-empty, use_semantic MUST be true.\n"
            "- If intent is 'search' and hybrid enforcement is enabled, use_cf MUST be true.\n"
            "- If both tools are used, weights MUST sum to 1.\n"
            "- Avoid setting weight_cf to 0 when hybrid enforcement is enabled.\n"
        )

        schema_hint = {
            "use_cf": True,
            "use_semantic": True,
            "weight_cf": 0.4,
            "weight_semantic": 0.6,
            "trace": [],
        }

        res = self.client.generate_json(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            force_json=True,
            schema_hint=schema_hint,
            max_retries=1,
        )

        obj = res["data"] or {}
        trace = list(obj.get("trace", [])) if isinstance(obj.get("trace"), list) else []

        # Defaults
        use_semantic = bool(obj.get("use_semantic", bool(query)))
        use_cf = bool(obj.get("use_cf", True))

        # Enforce query -> semantic
        if query:
            use_semantic = True

        # Hybrid enforcement for search
        if self.cfg.enforce_hybrid_for_search and intent == "search":
            use_cf = True
            trace.append("Hybrid enforcement: search requires CF + semantic")

        # Weights
        w_cf = float(obj.get("weight_cf", 0.4 if use_cf else 0.0))
        w_sem = float(obj.get("weight_semantic", 0.6 if use_semantic else 0.0))

        if use_cf and use_semantic:
            # Enforce minimum CF share if enabled
            if self.cfg.enforce_hybrid_for_search and intent == "search":
                w_cf = max(w_cf, float(self.cfg.min_cf_weight))
                w_sem = max(0.0, 1.0 - w_cf)

            # Normalize
            s = max(w_cf + w_sem, 1e-6)
            w_cf, w_sem = w_cf / s, w_sem / s

        elif use_cf and not use_semantic:
            w_cf, w_sem = 1.0, 0.0

        elif use_semantic and not use_cf:
            # If hybrid enforcement is on and intent is search, this should not happen
            if self.cfg.enforce_hybrid_for_search and intent == "search":
                use_cf = True
                w_cf = float(self.cfg.min_cf_weight)
                w_sem = 1.0 - w_cf
                trace.append("Corrected: CF re-enabled for hybrid search")
            else:
                w_cf, w_sem = 0.0, 1.0

        else:
            # Never allow neither
            use_cf, use_semantic = True, bool(query)
            w_cf, w_sem = (1.0, 0.0) if not query else (0.4, 0.6)
            trace.append("Corrected: at least one tool required")

        trace.extend(res.get("trace", []))
        trace.append("openai_planner_v2")

        out = {
            "use_cf": use_cf,
            "use_semantic": use_semantic,
            "weight_cf": float(w_cf),
            "weight_semantic": float(w_sem),
            "trace": trace,
        }

        self._log(f"Plan: use_cf={use_cf}, use_semantic={use_semantic}, w_cf={out['weight_cf']:.2f}, w_sem={out['weight_semantic']:.2f}")
        return out


Writing agents/planner_agent.py


In [14]:
%%writefile agents/critic_agent.py
from __future__ import annotations
from typing import Any, Dict, List
from config import POCConfig
from agents.openai_client import OpenAIJSONClient


class CriticAgent:
    """
    Critic v2:
    - Enforces hard safety / suitability constraints
    - Evaluates novelty & popularity
    - Checks genre diversity
    - Can request advantage-weight + novelty adjustments
    """

    def __init__(self, cfg: POCConfig):
        self.cfg = cfg
        self.client = OpenAIJSONClient(
            model="gpt-4o-mini",
            temperature=0.2,
            verbose=bool(cfg.verbose_openai),
        )
        self.verbose = bool(cfg.verbose)

    def _log(self, msg: str):
        if self.verbose:
            print(f"[CriticAgent] {msg}")

    def _genre_diversity(self, recs: List[Dict[str, Any]], topn: int) -> int:
        genres = set()
        for r in recs[:topn]:
            g = (r.get("genres") or "")
            for token in g.split("|"):
                t = token.strip()
                if t:
                    genres.add(t)
        return len(genres)

    def _mean_popularity(self, recs: List[Dict[str, Any]], topn: int) -> float:
        vals = []
        for r in recs[:topn]:
            pop = ((r.get("signals") or {}).get("baseline_popularity"))
            if isinstance(pop, (int, float)):
                vals.append(float(pop))
        return float(sum(vals) / max(len(vals), 1))

    def run(
        self,
        intent: str,
        context: Dict[str, Any],
        recs: List[Dict[str, Any]],
    ) -> Dict[str, Any]:

        intent = (intent or "explore").strip().lower()
        topn = int(self.cfg.critic_topn)

        needs_rerank = False
        adjustments: Dict[str, Any] = {}
        trace: List[str] = []

        # 1) DETERMINISTIC GUARDRAILS

        mean_pop = self._mean_popularity(recs, topn)
        if mean_pop > float(self.cfg.popularity_mean_threshold):
            needs_rerank = True
            adjustments["novelty_lambda"] = min(
                0.35, float(self.cfg.novelty_lambda) + 0.10
            )
            trace.append(f"Top-{topn} mean popularity too high: {mean_pop:.2f}")

        gdiv = self._genre_diversity(recs, topn)
        if gdiv < int(self.cfg.genre_diversity_min_unique):
            needs_rerank = True
            adjustments["diversity_boost"] = 0.10
            trace.append(f"Genre diversity too low: unique_genres={gdiv}")

        # ======================================================
        # 2) HARD MATURITY / SUITABILITY VETO  (OPTION B)
        # ======================================================

        genres: List[str] = []
        for r in recs[:topn]:
            g = r.get("genres", "")
            if isinstance(g, str):
                genres.extend([x.strip() for x in g.split("|")])

        children_ratio = (
            sum(1 for g in genres if g.lower() == "children")
            / max(1, len(genres))
        )

        user_query = (context.get("query") or "").lower()
        children_allowed = any(
            kw in user_query
            for kw in ["kid", "kids", "family", "children", "child"]
        )

        if children_ratio > 0.30 and not children_allowed:
            needs_rerank = True
            adjustments["exclude_genres"] = ["Children"]
            trace.append("hard_veto_children_content")

        # 3) LLM CRITIQUE

        system_prompt = (
            "You are a recommendation critic for a movie recommender.\n"
            "Return ONLY valid JSON.\n"
            "Do NOT include commentary.\n"
            "Do NOT reveal hidden reasoning.\n"
        )

        user_prompt = (
            f"Intent: {intent}\n"
            f"User context: {context}\n\n"
            "You will be given the top recommendations with optional signals.\n"
            "Assess:\n"
            "- Does the list satisfy the query and constraints?\n"
            "- Is novelty too low (too popular)?\n"
            "- Is the list too narrow in genres?\n"
            "- Are advantage signals reflected in top-ranked items?\n\n"
            "Return JSON with exactly these keys:\n"
            "{\n"
            '  "needs_rerank": boolean,\n'
            '  "adjustments": object,\n'
            '  "trace": array of strings\n'
            "}\n\n"
            f"Top recommendations:\n{recs[:10]}\n"
        )

        schema_hint = {
            "needs_rerank": needs_rerank,
            "adjustments": adjustments,
            "trace": trace,
        }

        res = self.client.generate_json(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            force_json=True,
            schema_hint=schema_hint,
            max_retries=1,
        )

        data = res.get("data") or {}
        llm_needs = bool(data.get("needs_rerank", False))
        llm_adj = data.get("adjustments", {})
        llm_trace = data.get("trace", [])

        # 4) MERGE DECISIONS (HARD RULES WIN)

        needs_rerank = bool(needs_rerank or llm_needs)
        if isinstance(llm_adj, dict):
            adjustments.update(llm_adj)

        if isinstance(llm_trace, list):
            trace.extend(llm_trace)

        trace.extend(res.get("trace", []))
        trace.append("openai_critic_v2")

        out = {
            "needs_rerank": needs_rerank,
            "adjustments": adjustments,
            "trace": trace,
        }

        self._log(
            f"Critic: needs_rerank={needs_rerank}, adjustments={adjustments}"
        )
        return out


Writing agents/critic_agent.py


In [15]:
%%writefile agents/explainer_agent.py
import json
from typing import Any, Dict, List, Optional

from agents.openai_client import OpenAIJSONClient


class ExplainerAgent:
    """
    OpenAI-backed explanation agent.

    Design goals:
    - Conversational and user-facing
    - Explicitly tied to the user's stated constraints
    - Honest about strength of fit (no forced justification)
    - Grounded in titles/genres only (no invented plot facts)
    - No algorithm, model, or scoring language
    """

    def __init__(self, model: str = "gpt-4o-mini"):
        self.llm = OpenAIJSONClient(model=model, temperature=0.55)

    def run(
        self,
        intent: str,
        strategy: Dict[str, Any],
        recs: List[Dict[str, Any]],
        context: Optional[Dict[str, Any]] = None,
    ) -> Dict[str, Any]:
        context = context or {}

        system_prompt = """
You are the Netflix Explainer.

Your job is to explain WHY these recommendations fit the user's request —
without exaggeration and without forcing relevance.

CORE PRINCIPLES:
- Be honest about fit quality. Not every recommendation is a perfect match.
- Emphasize the strongest matches first.
- If a title is only a partial or tonal match, say so gently.
- Never invent plot details or specific scenes.

RULES:
- Speak directly TO the user ("you"), never AS the user.
- Do NOT mention algorithms, models, embeddings, rankings, or scores.
- Do NOT claim deep romance, realism, or emotional depth unless the title clearly supports it.
- Use careful language when appropriate:
  "leans more toward", "has elements of", "is less about romance but shares a grounded tone".
- Explicitly reference the user's constraints (e.g., "feels real", "not cheesy", "grounded", "emotionally honest").
- Avoid generic genre-only explanations.
- Output ONLY valid JSON (no markdown, no commentary).

TONE:
- Calm, editorial, and human.
- Similar to Netflix's in-app editorial blurbs.
"""

        # Keep explanation grounded: title + genres only
        top3 = [
            {
                "title": r.get("title", ""),
                "genres": r.get("genres", ""),
            }
            for r in (recs or [])[:3]
        ]

        user_prompt = f"""
User intent:
{intent}

User context (JSON):
{json.dumps(context, indent=2)}

Top recommendations (title + genres only):
{json.dumps(top3, indent=2)}

WRITE:
- one_liner:
  ONE sentence summarizing why these picks were chosen,
  explicitly referencing the user's constraints.

- bullets:
  EXACTLY 3 bullets.
  Each bullet must:
  - Mention a specific title.
  - Explain how it aligns with the user's constraints.
  - Use honest framing (strong match vs partial/tonal match).
  - Avoid plot claims or factual specifics.

IMPORTANT:
- If a movie is not clearly romantic or emotionally grounded,
  describe it as a tonal or adjacent match rather than a direct one.
- Do NOT try to justify poor matches.

RETURN ONLY THIS JSON SCHEMA:
{{
  "one_liner": "string",
  "bullets": ["string", "string", "string"]
}}
"""

        schema_hint = {
            "one_liner": "These films focus on grounded, emotionally honest storytelling rather than glossy or exaggerated romance.",
            "bullets": [
                "Get Real (1998) is often described as emotionally raw and understated, which aligns well with a preference for romance that feels genuine rather than performative.",
                "All the Real Girls (2003) leans into quiet, character-driven moments, favoring emotional realism over dramatic spectacle.",
                "Blue Valentine (2010) is known for its unvarnished tone, offering a more difficult but deeply authentic take on romantic relationships.",
            ],
        }

        resp = self.llm.generate_json(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            force_json=True,
            schema_hint=schema_hint,
            max_retries=1,
        )

        data = resp["data"] if resp["ok"] else {}

        one_liner = data.get("one_liner", "")
        bullets = data.get("bullets", [])

        if not isinstance(one_liner, str):
            one_liner = ""

        if not isinstance(bullets, list):
            bullets = []

        bullets = [b for b in bullets if isinstance(b, str)]

        # Ensure exactly 3 bullets (safe padding)
        while len(bullets) < 3:
            bullets.append("")
        bullets = bullets[:3]

        return {
            "one_liner": one_liner,
            "bullets": bullets,
        }


Writing agents/explainer_agent.py


In [16]:
%%writefile graph/orchestrator.py
from langgraph.graph import StateGraph, START, END
from typing import Dict, Any


class AgenticGraph:
    """
    LangGraph orchestrator (v1/v2 compatible).

    Key points:
    - uses intent_obj key
    - trace_log is appended at each node
    - supports critic-driven rerank adjustments
    """

    def __init__(self, agents, tools, ranker, cfg):
        self.agents = agents
        self.tools = tools
        self.ranker = ranker
        self.cfg = cfg

    def build(self):
        graph = StateGraph(dict)

        def _append_trace(state: Dict[str, Any], event: Dict[str, Any]) -> Dict[str, Any]:
            trace = list(state.get("trace_log", []))
            trace.append(event)
            return {**state, "trace_log": trace}

        def intent_node(state: Dict[str, Any]) -> Dict[str, Any]:
            intent_obj = self.agents["intent"].run(state["context"])
            state2 = {**state, "intent_obj": intent_obj}
            return _append_trace(
                state2,
                {
                    "node": "intent",
                    "intent": intent_obj.get("intent"),
                    "confidence": intent_obj.get("confidence"),
                    "trace": intent_obj.get("trace", []),
                },
            )

        def plan_node(state: Dict[str, Any]) -> Dict[str, Any]:
            intent = (state.get("intent_obj") or {}).get("intent", "explore")
            plan = self.agents["planner"].run(intent, state["context"])
            state2 = {**state, "plan": plan}
            return _append_trace(
                state2,
                {
                    "node": "plan",
                    "use_cf": plan.get("use_cf"),
                    "use_semantic": plan.get("use_semantic"),
                    "weights": {"cf": plan.get("weight_cf"), "semantic": plan.get("weight_semantic")},
                },
            )

        def retrieve_node(state: Dict[str, Any]) -> Dict[str, Any]:
            updates = {}
            plan = state.get("plan") or {}
            use_cf = bool(plan.get("use_cf", True))
            use_sem = bool(plan.get("use_semantic", False))

            if use_cf:
                updates["cf"] = self.tools["cf"].recommend(state["user_id"], self.cfg.cf_k)

            if use_sem:
                query = (state["context"].get("query") or "").strip()
                if query:
                    updates["sem"] = self.tools["semantic"].search(query, self.cfg.semantic_k)
                else:
                    updates["sem"] = []

            state2 = {**state, **updates}
            return _append_trace(
                state2,
                {
                    "node": "retrieve",
                    "use_cf": use_cf,
                    "use_semantic": use_sem,
                    "cf_rows": int(len(state2.get("cf"))) if state2.get("cf") is not None else 0,
                    "sem_rows": int(len(state2.get("sem"))) if state2.get("sem") is not None else 0,
                },
            )

        def rank_node(state: Dict[str, Any]) -> Dict[str, Any]:
            recs = self.ranker(state)
            state2 = {**state, "recs": recs}
            top = recs[0]["title"] if recs else None
            return _append_trace(state2, {"node": "rank", "top1": top, "recs_count": len(recs)})

        def critic_node(state: Dict[str, Any]) -> Dict[str, Any]:
            intent = (state.get("intent_obj") or {}).get("intent", "explore")
            critic = self.agents["critic"].run(intent, state["context"], state.get("recs", []))
            state2 = {**state, "critic": critic}
            return _append_trace(
                state2,
                {
                    "node": "critic",
                    "needs_rerank": critic.get("needs_rerank"),
                    "adjustments": critic.get("adjustments", {}),
                },
            )

        def should_rerank(state: Dict[str, Any]) -> str:
            critic = state.get("critic") or {}
            return "rerank" if bool(critic.get("needs_rerank", False)) else "explain"

        def rerank_node(state: Dict[str, Any]) -> Dict[str, Any]:
            """
            Apply critic adjustments in-state and re-rank once.
            We do NOT mutate global cfg; we only tweak state knobs.
            """
            critic = state.get("critic") or {}
            adj = critic.get("adjustments") or {}

            # Store knobs in state for ranker_v2 to read if you want to extend;
            # For now, rankers use cfg, so we adjust plan weights only (safe).
            plan = dict(state.get("plan") or {})
            w_cf = float(plan.get("weight_cf", 0.4))
            w_sem = float(plan.get("weight_semantic", 0.6))

            w_cf_delta = float(adj.get("weight_cf_delta", 0.0))
            w_sem_delta = float(adj.get("weight_semantic_delta", 0.0))

            w_cf = max(0.0, min(1.0, w_cf + w_cf_delta))
            w_sem = max(0.0, min(1.0, w_sem + w_sem_delta))

            if w_cf > 0 and w_sem > 0:
                s = max(w_cf + w_sem, 1e-6)
                w_cf, w_sem = w_cf / s, w_sem / s
            elif w_cf > 0:
                w_cf, w_sem = 1.0, 0.0
            else:
                w_cf, w_sem = 0.0, 1.0

            plan["weight_cf"] = float(w_cf)
            plan["weight_semantic"] = float(w_sem)
            plan_trace = list(plan.get("trace", [])) if isinstance(plan.get("trace", []), list) else []
            plan_trace.append("critic_rerank_applied")
            plan["trace"] = plan_trace

            state2 = {**state, "plan": plan}

            # Re-rank once
            recs = self.ranker(state2)
            state3 = {**state2, "recs": recs}

            return _append_trace(
                state3,
                {
                    "node": "rerank",
                    "applied_adjustments": adj,
                    "new_weights": {"cf": plan["weight_cf"], "semantic": plan["weight_semantic"]},
                    "top1": recs[0]["title"] if recs else None,
                },
            )

        def explain_node(state: Dict[str, Any]) -> Dict[str, Any]:
            intent = (state.get("intent_obj") or {}).get("intent", "explore")
            explanation = self.agents["explainer"].run(intent, state.get("plan", {}), state.get("recs", []))
            state2 = {**state, "explanation": explanation}
            return _append_trace(state2, {"node": "explain", "one_liner": explanation.get("one_liner", "")})

        graph.add_node("intent", intent_node)
        graph.add_node("plan", plan_node)
        graph.add_node("retrieve", retrieve_node)
        graph.add_node("rank", rank_node)
        graph.add_node("critic", critic_node)
        graph.add_node("rerank", rerank_node)
        graph.add_node("explain", explain_node)

        graph.add_edge(START, "intent")
        graph.add_edge("intent", "plan")
        graph.add_edge("plan", "retrieve")
        graph.add_edge("retrieve", "rank")
        graph.add_edge("rank", "critic")
        graph.add_conditional_edges("critic", should_rerank, {"rerank": "rerank", "explain": "explain"})
        graph.add_edge("rerank", "explain")
        graph.add_edge("explain", END)

        return graph.compile()


Writing graph/orchestrator.py


In [17]:
%%writefile agents/openai_client.py
import json
from typing import Any, Dict, Optional, Tuple
from openai import OpenAI


class OpenAIJSONClient:
    """
    OpenAI helper with:
    - JSON-only enforcement
    - bounded repair
    - structured verbosity (NO chain-of-thought)
    """

    def __init__(
        self,
        model: str = "gpt-4o-mini",
        temperature: float = 0.2,
        timeout: Optional[float] = None,
        verbose: bool = False,
    ):
        self.client = OpenAI()
        self.model = model
        self.temperature = temperature
        self.timeout = timeout
        self.verbose = verbose

    def _log(self, msg: str):
        if self.verbose:
            print(f"[OpenAIJSONClient] {msg}")

    def _call(self, system_prompt: str, user_prompt: str, force_json: bool = True) -> str:
        self._log("Calling OpenAI API")

        kwargs: Dict[str, Any] = {}
        if force_json:
            kwargs["response_format"] = {"type": "json_object"}

        resp = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=[
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": user_prompt.strip()},
            ],
            **kwargs,
        )

        raw = resp.choices[0].message.content.strip()
        self._log(f"Raw response length: {len(raw)} chars")
        return raw

    @staticmethod
    def _try_parse(raw: str) -> Tuple[Optional[Dict[str, Any]], bool]:
        if not raw or not raw.strip():
            return None, False

        try:
            return json.loads(raw), True
        except Exception:
            start = raw.find("{")
            end = raw.rfind("}")
            if start != -1 and end != -1 and end > start:
                try:
                    return json.loads(raw[start:end + 1]), True
                except Exception:
                    return None, False
        return None, False

    def generate_json(
        self,
        system_prompt: str,
        user_prompt: str,
        force_json: bool = True,
        schema_hint: Optional[Dict[str, Any]] = None,
        max_retries: int = 1,
    ) -> Dict[str, Any]:
        """
        Returns:
        {
            "ok": bool,
            "data": dict,
            "raw": str,
            "trace": list[str]
        }
        """

        trace = []
        raw = ""

        # Attempt 1
        try:
            raw = self._call(system_prompt, user_prompt, force_json)
            obj, ok = self._try_parse(raw)
            if ok and isinstance(obj, dict):
                self._log("JSON parsed successfully")
                return {"ok": True, "data": obj, "raw": raw, "trace": trace}
            trace.append("json_parse_failed")
            self._log("JSON parse failed")
        except Exception as e:
            trace.append(f"openai_call_failed:{type(e).__name__}")
            self._log(f"OpenAI call failed: {e}")

        # Repair attempt
        for _ in range(max_retries):
            repair_system = (
                "You repair invalid JSON.\n"
                "Return ONLY valid JSON.\n"
                "Do NOT include commentary.\n"
            )

            repair_user = (
                "The previous output was not valid JSON.\n"
                "Fix it and return ONLY JSON.\n"
                "If content is missing, use safe defaults.\n"
            )

            if schema_hint:
                repair_user += f"\nJSON schema hint:\n{json.dumps(schema_hint, indent=2)}\n"

            repair_user += f"\nInvalid output:\n{raw}\n"

            try:
                self._log("Attempting JSON repair")
                raw2 = self._call(repair_system, repair_user, force_json)
                obj2, ok2 = self._try_parse(raw2)
                if ok2 and isinstance(obj2, dict):
                    trace.append("json_repair_success")
                    self._log("JSON repair succeeded")
                    return {"ok": True, "data": obj2, "raw": raw2, "trace": trace}
                trace.append("json_repair_failed")
                raw = raw2
            except Exception as e:
                trace.append(f"openai_repair_failed:{type(e).__name__}")
                self._log(f"Repair failed: {e}")

        self._log("Falling back to safe empty JSON")
        return {"ok": False, "data": {}, "raw": raw, "trace": trace}


Writing agents/openai_client.py


In [18]:
# AGENTIC RECOMMENDER — FULL LOCAL DEMO
import json
import pandas as pd
from config import POCConfig
from tools.data_loader import MovieLensLoader
from tools.cf_tool import SimpleCFRecommender
from tools.semantic_tool import SemanticSearchTool
from agents.intent_agent import IntentAgent
from agents.planner_agent import PlannerAgent
from agents.critic_agent import CriticAgent
from agents.explainer_agent import ExplainerAgent
from graph.orchestrator import AgenticGraph
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# 1. CONFIG
cfg = POCConfig()

# 2. DATA LOADING (SAFE)
print("Loading data...")
ratings, movies = MovieLensLoader(cfg).load()
print(f"Ratings: {len(ratings):,}, Movies: {len(movies):,}")

# 3. COLLABORATIVE FILTERING
print("Training CF model...")
cf_tool = SimpleCFRecommender()
cf_tool.fit(ratings)

# 4. SEMANTIC SEARCH (PERSISTED)
print("Preparing semantic index...")
semantic_tool = SemanticSearchTool(cfg)
semantic_tool.build_or_load(movies)

# 5. LOCAL LLM (CONTROL AGENTS)
print("Loading local LLM...")
tokenizer = AutoTokenizer.from_pretrained(cfg.llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    cfg.llm_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
llm_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=cfg.llm_max_new_tokens,
    temperature=cfg.llm_temperature,
)

class LocalLLM:
    def generate_json(self, prompt: str):
        out = llm_pipe(prompt, return_full_text=False)[0]["generated_text"]

        if not out or not out.strip():
            return {
                "parse_error": True,
                "raw": "",
                "trace": ["empty_llm_output"]
            }

        # Try direct JSON
        try:
            return json.loads(out)
        except Exception:
            pass

        # Try extracting first {...}
        start = out.find("{")
        end = out.rfind("}")

        if start != -1 and end != -1 and end > start:
            try:
                return json.loads(out[start:end + 1])
            except Exception:
                pass

        # Final safe fallback
        return {
            "parse_error": True,
            "raw": out,
            "trace": ["json_parse_failed"]
        }

llm = LocalLLM()
# 6. AGENTS
intent_agent = IntentAgent()
planner_agent = PlannerAgent(cfg)
critic_agent = CriticAgent(cfg)
explainer_agent = ExplainerAgent()

agents = {
    "intent": intent_agent,
    "planner": planner_agent,
    "critic": critic_agent,
    "explainer": explainer_agent,
}

tools = {
    "cf": cf_tool,
    "semantic": semantic_tool,
}

# 7. RANKER (v1 / v2 SWITCH)
from tools.item_stats import ItemStats
from rankers.ranker_v1 import RankerV1
from rankers.ranker_v2 import RankerV2

movie_map = movies.set_index("movieId")[["title", "genres"]].to_dict("index")

# Build baseline statistics ONCE
item_stats = ItemStats.build(ratings)

if getattr(cfg, "strategy_version", "v2") == "v1":
    print("Using Ranker v1 (baseline)")
    ranker = RankerV1(cfg, movie_map)
else:
    print("Using Ranker v2 (hybrid + advantage-weighted)")
    ranker = RankerV2(cfg, movie_map, item_stats)


# ---------------------------
# 8. BUILD GRAPH
# ---------------------------
graph = AgenticGraph(
    agents=agents,
    tools=tools,
    ranker=ranker,
    cfg=cfg
).build()

# ---------------------------
# 9. RUN DEMO QUERY
# ---------------------------
demo_input = {
    "user_id": 7,
    "context": {
        "query": "romantic movie that feels real, not cheesy",
        "available_minutes": 120,
        "novelty_tolerance": 0.4
    }
}

print("\nRUNNING AGENTIC RECOMMENDATION...\n")
print("TRACE: invoking graph with keys:", list(demo_input.keys()))
print("TRACE: demo_input =", demo_input)
result = graph.invoke(demo_input)
print("TRACE: result keys:", list(result.keys()))

# ---------------------------
# 10. DISPLAY RESULTS
# ---------------------------
print("INTENT")
print(json.dumps(result["intent_obj"], indent=2))

print("\nSTRATEGY")
print(json.dumps(result["plan"], indent=2))

print("\nTOP RECOMMENDATIONS")
for i, r in enumerate(result["recs"], 1):
    print(f"{i:>2}. {r['title']}  ({r['genres']})")

print("\nEXPLANATION")
print(json.dumps(result["explanation"], indent=2))

print("\nAGENT TRACE (ABRIDGED)")
print("\nTRACE LOG (per node)")
print(json.dumps(result.get("trace_log", []), indent=2))

print("\nINTENT")
print(json.dumps(result["intent_obj"], indent=2))

print("\nPLAN")
print(json.dumps(result["plan"], indent=2))

print("\nCRITIC")
print(json.dumps(result["critic"], indent=2))

Loading data...
MovieLens dataset not found. Downloading...
Download and extraction complete.
Ratings: 100,836, Movies: 9,742
Training CF model...
Preparing semantic index...
Building embeddings (one-time)
Loading local LLM...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


Using Ranker v2 (hybrid + advantage-weighted)

RUNNING AGENTIC RECOMMENDATION...

TRACE: invoking graph with keys: ['user_id', 'context']
TRACE: demo_input = {'user_id': 7, 'context': {'query': 'romantic movie that feels real, not cheesy', 'available_minutes': 120, 'novelty_tolerance': 0.4}}
[OpenAIJSONClient] Calling OpenAI API
[OpenAIJSONClient] Raw response length: 105 chars
[OpenAIJSONClient] JSON parsed successfully
[PlannerAgent] Plan: use_cf=True, use_semantic=True, w_cf=0.50, w_sem=0.50
[OpenAIJSONClient] Calling OpenAI API
[OpenAIJSONClient] Raw response length: 440 chars
[OpenAIJSONClient] JSON parsed successfully
[CriticAgent] Critic: needs_rerank=True, adjustments={'remove': [], 'add': []}
TRACE: result keys: ['user_id', 'context', 'intent_obj', 'trace_log', 'plan', 'cf', 'sem', 'recs', 'critic', 'explanation']
INTENT
{
  "intent": "search",
  "confidence": 1.0,
  "needs_clarification": false,
  "clarification_question": "",
  "trace": [
    "user has a specific query for a

In [19]:
from google.colab import files

files.download("embeddings/movie_embeddings.npy")
files.download("embeddings/movie_ids.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>